<a href="https://colab.research.google.com/github/IgorLytkin/MachineLearning/blob/main/Hometask_%D0%94%D0%B5%D1%82%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_%D1%80%D0%B0%D0%B7%D0%B3%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D0%BD%D1%8B%D1%85_%D1%81%D0%BE%D0%BE%D0%B1%D1%89%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install pymorphy2
!pip install razdel

import nltk
nltk.download('stopwords')

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 20.2 MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import os
import glob
import pymorphy2
from razdel import tokenize
from tqdm import tqdm
tqdm.pandas()
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn import model_selection
from sklearn import pipeline
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV

import matplotlib
from matplotlib import pyplot as plt
matplotlib.pyplot.style.use('ggplot')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Загрузка News

##Загрузка marked_news

In [ ]:
marked_news = pd.read_csv("/content/drive/My Drive/Fraud_Finance_Data/News/marked_news.csv"
                            , delimiter=';'
                            # , nrows=5
                          )
marked_news  = marked_news.rename(columns={"class": "target"})

In [ ]:
marked_news.shape

(9404, 14)

In [ ]:
marked_news.head(3)

,channel,channel_id,members,post_id,date,url,views,cover_ratio,ticker,text,author,likes,tickers,class
0,MarketTwits,MarketTwits,137000,27956,2019.01.18 18:30:00,t.me/markettwits/27956,5300,0.04,$RS,🔥 ❗️ 🇨🇳 🇺🇸 #торговыевойныкитай Китай обещает у...,NaN,NaN,#EA-RM #GPS-RM #OBUV #TORS,0
1,ФинДип,findeep,809,569,2019.01.22 10:39:27,t.me/FinDeep/569,165,0.20,NaN,Франк может подешеветь на фоне опасений Нацбан...,NaN,NaN,#OBUV #ROSB #TANL,0
2,Криптокритика / Cryptocritique,cryptocritique,6240,2877,2019.01.22 16:06:36,t.me/cryptocritique/2877,18,0.00,NaN,"Сюжет ТВЦ о проекте, «который позволяет его уч...",NaN,NaN,#LIFE #OBUV,0


In [ ]:
marked_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404 entries, 0 to 9403
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   channel      9404 non-null   object 
 1   channel_id   9404 non-null   object 
 2   members      9404 non-null   int64  
 3   post_id      9404 non-null   int64  
 4   date         9404 non-null   object 
 5   url          9404 non-null   object 
 6   views        9404 non-null   int64  
 7   cover_ratio  9404 non-null   float64
 8   ticker       4376 non-null   object 
 9   text         9404 non-null   object 
 10  author       0 non-null      float64
 11  likes        0 non-null      float64
 12  tickers      9397 non-null   object 
 13  class        9404 non-null   int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 1.0+ MB


In [ ]:
marked_news.groupby(['channel']).size()
# df1.set_option("display.max_rows", 5)
# sort_values('channel', ascending = True)

channel
BCS Express                                131
Coin Post – Деньги, инвестиции, биткоин     60
Finam Alert                                211
FinanceMarker.ru                            99
Forbes Russia                              448
                                          ... 
ФИНАНСОВЫЕ ДУМКИ                            56
ФинДип                                     816
Финансовая Независимость                    20
Экономика                                  251
ЮРИСТ ЕВГЕНИЙ ШУПИКОВ                       13
Length: 61, dtype: int64

In [ ]:
#marked_news[marked_news.channel == 'ЗЛОЙ ФИНАНСИСТ 😡'].groupby(['channel']).apply(lambda x: x.sort_values('class', ascending=False))

In [ ]:
marked_news.groupby(['channel']).agg({'channel_id': 'count', 'class': 'sum'}).sort_values(by = ['class'], ascending=False).head(5)

,channel_id,class
channel,,
РынкиДеньгиВласть | РДВ,809,169
Сигналы РЦБ,708,118
ЗЛОЙ ФИНАНСИСТ 😡,146,95
СМАРТЛАБ,289,27
MarketTwits,1226,19


##Загрузка results_news_authors_likes

In [ ]:
results_news_authors_likes = pd.read_csv("/content/drive/My Drive/Fraud_Finance_Data/News/results_news_authors_likes.csv"
                            , delimiter=';'
                            # , quoting = 2
                            # , skiprows = [0, 512648]
                            # , nrows=5
                            # , on_bad_lines = 'warn'
                            , error_bad_lines = False
                          )

b'Skipping line 512649: expected 13 fields, saw 14\n'
b'Skipping line 537906: expected 13 fields, saw 14\n'


In [ ]:
results_news_authors_likes.head(2)

,channel,channel_id,members,post_id,date,url,views,cover_ratio,ticker,text,author,likes,tickers
0,ФИНАНСОВЫЕ ДУМКИ,marketdumki,46800,2504,2021.04.19 13:13:28,t.me/MarketDumki/2504,18700,0.4,NaN,"Отличное нововведение , на которое давно был з...",NaN,NaN,#OBUV #ROSB
1,ФИНАНСОВЫЕ ДУМКИ,marketdumki,46800,2505,2021.04.19 16:16:19,t.me/MarketDumki/2505,18800,0.4,NaN,''Ну никак пока не могут вырасти акции Газпром...,NaN,NaN,#GAZA #GAZAP #GAZP #OBUV #ROSB


##Загрузка forums_utf

In [ ]:
forums_utf = pd.read_csv("/content/drive/My Drive/Fraud_Finance_Data/News/forums_utf.csv"
                            , delimiter=';'
                            # , quoting = 2
                            # , skiprows = [0, 512648]
                            # , nrows=5
                            # , on_bad_lines = 'warn'
                            , error_bad_lines = False
                          )

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
forums_utf.tail(3)

,Channel,Channel_id,members,post_id,date,url,views,cover_ratio,ticker,text,author,likes
2533915,Смартлаб,smart_lab,NaN,9327205,2019.01.18 13:13:47,https://smart-lab.ru/forum/ZVEZ/page53/,NaN,NaN,#ZVEZ,"Будут новости большие в понедельник уже, Совет...",сергей,0
2533916,Смартлаб,smart_lab,NaN,9320930,2019.01.17 10:33:38,https://smart-lab.ru/forum/ZVEZ/page53/,NaN,NaN,#ZVEZ,Акция созрела для скачка однозначно.,сергей,0
2533917,Смартлаб,smart_lab,NaN,9270027,2019.01.03 11:27:30,https://smart-lab.ru/forum/ZVEZ/page53/,NaN,NaN,#ZVEZ,Скупка с самого начала года. Хороший знак.,Уоррен Баффет,0


#Загрузка Prices

In [ ]:
# os.chdir("/content/drive/My Drive/Fraud_Finance_Data/Prices/2019-2020/")
# path = os.getcwd()
# csv_files = glob.glob(os.path.join(path, "*.csv"))
# print(csv_files)

['/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/ABRD.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/LKOH.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/AFKS.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/AFLT.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/AGRO.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/AKRN.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/ALBK.csv', '/content/drive/.shortcut-targets-by-id/15I37gBpPgYtX7PojExDuiymMuq3tndBF/Fraud_Finance_Data/Prices/2019-2020/ALNU.csv', '/content/drive/.shortcut-targe

In [ ]:
prices = []

os.chdir("/content/drive/My Drive/Fraud_Finance_Data/Prices/2019-2020/")
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

for filename in csv_files:
    df_next = pd.read_csv(filename, index_col=None, header=0)
    prices.append(df_next)


os.chdir("/content/drive/My Drive/Fraud_Finance_Data/Prices/2021/")
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

for filename in csv_files:
    df_next = pd.read_csv(filename, index_col=None, header=0)
    prices.append(df_next)

prices_all = pd.concat(prices, axis=0, ignore_index=True)

In [ ]:
prices_all.tail(3)
#28688806

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
28688804,ZVEZ,1,24.09.2021,14:10:00,4.270,4.270,4.270,4.270,1000
28688805,ZVEZ,1,24.09.2021,17:14:00,4.205,4.205,4.205,4.205,4000
28688806,ZVEZ,1,24.09.2021,18:45:00,4.290,4.290,4.290,4.290,2000


#Загрузка marked_news

In [ ]:
marked_news = pd.read_csv("/content/drive/My Drive/Fraud_Finance_Data/News/marked_news.csv"
                            , delimiter=';'
                            # , nrows=5
                          )
marked_news = marked_news.rename(columns={"class": "target"})
marked_news = marked_news.drop_duplicates()

##Pre-clearing

In [ ]:
morph = pymorphy2.MorphAnalyzer() #for lammatization
def preprocessing(s):
  s = s.lower()
  s = tokenize(s)
  s = [i.text for i in s]
  lemmas = []
  for i in s:
    lemmas.append( morph.parse(i)[0].normal_form )
  return lemmas

##Cleaning

In [ ]:
marked_news['text_cleaning'] = marked_news.text.progress_apply(preprocessing)


100%|██████████| 7668/7668 [04:10<00:00, 30.60it/s]


In [ ]:
marked_news['text_union'] = marked_news.text_cleaning.progress_apply(lambda x: " ".join(x))

100%|██████████| 7668/7668 [00:00<00:00, 96264.07it/s]


In [ ]:
marked_news[['text', 'text_cleaning', 'text_union']].head(4)

,text,text_cleaning,text_union
0,🔥 ❗️ 🇨🇳 🇺🇸 #торговыевойныкитай Китай обещает у...,"[🔥, ❗️, 🇨🇳, 🇺🇸, #, торговыевойныкитай, китай, ...",🔥 ❗️ 🇨🇳 🇺🇸 # торговыевойныкитай китай обещать ...
1,Франк может подешеветь на фоне опасений Нацбан...,"[франк, мочь, подешеветь, на, фон, опасение, н...",франк мочь подешеветь на фон опасение нацбанк ...
2,"Сюжет ТВЦ о проекте, «который позволяет его уч...","[сюжет, твц, о, проект, ,, «, который, позволя...","сюжет твц о проект , « который позволять он уч..."
3,Boeing представил свой прототип летающего авто...,"[boeing, представить, свой, прототип, летать, ...",boeing представить свой прототип летать автомо...


In [ ]:
train, test  = train_test_split(marked_news, random_state=1, test_size=0.2)

In [ ]:
test.head(2)

,channel,channel_id,members,post_id,date,url,views,cover_ratio,ticker,text,author,likes,tickers,target,text_cleaning,text_union
2725,Дивиденды Forever,divForever,69000,4535,2020.02.27 17:13:39,t.me/divForever/4535,18,0.0,$GMKN,''#GMKN Норильский никель. Обзор финансовых по...,NaN,NaN,#GMKN #HPQ-RM #OBUV #RDRB #ROSB #ZILL,0,"[', ', #, gmkn, норильский, никель, ., обзор, ...",' ' # gmkn норильский никель . обзор финансовы...
1305,Банкста,banksta,171000,6714,2019.12.18 15:39:19,t.me/banksta/6714,34900,0.2,NaN,''Сергей Янчуков - владелец золотодобывающей к...,NaN,NaN,#GAZA #GAZAP #OBUV #ROSB,0,"[', ', сергей, янчук, -, владелец, золотодобыв...",' ' сергей янчук - владелец золотодобывающий к...


##TF-IDF Vectorizer

In [ ]:
stopwords = set(nltk_stopwords.words('russian'))

tfidf_vectorizer = TfidfVectorizer(min_df=0.05, stop_words=stopwords)
processed_train_tfidf = tfidf_vectorizer.fit_transform(train.text_union)
processed_test_tfidf = tfidf_vectorizer.transform(test.text_union)
print(processed_train_tfidf[5])

  (0, 4)	0.795715477706835
  (0, 132)	0.6056706023390794


###GridSearchCV

In [ ]:
rf_model_tfidf_grid = RandomForestClassifier()

In [ ]:
kfold = KFold(n_splits=5)
grid = GridSearchCV(rf_model_tfidf_grid, cv=kfold,
                    param_grid=[{'n_estimators': [100, 200, 250, 50, 300], 
                                 'max_depth': [5, 10, 30, 50, 100, 200]}])
grid.fit(processed_train_tfidf, train.target)

In [ ]:
grid.best_params_

{'max_depth': 30, 'n_estimators': 50}

###RandomForestClassifier

In [ ]:
rf_model_tfidf = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=111, n_jobs=-1)
# rf_model_tfidf = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
# rf_model_tfidf = LogisticRegressionCV(solver='newton-cg')

rf_model_tfidf.fit(processed_train_tfidf, train.target)

train_predictions_tfidf = rf_model_tfidf.predict(processed_train_tfidf)
test_predictions_tfidf = rf_model_tfidf.predict(processed_test_tfidf)

# train_predictions_tfidf = rf_model_tfidf.predict_proba(processed_train_tfidf)
# test_predictions_tfidf = rf_model_tfidf.predict_proba(processed_test_tfidf)

In [ ]:
from collections import Counter
Counter(test_predictions_tfidf)

In [ ]:
print(accuracy_score(train_predictions_tfidf, train.target))
# print(accuracy_score(test_predictions_tfidf, test.target))

0.9905445060319531


In [ ]:
print(precision_score(test_predictions_tfidf, test.target))
print(recall_score(test_predictions_tfidf, test.target))
print(f1_score(test_predictions_tfidf, test.target))
print(log_loss(test_predictions_tfidf, test.target))

0.0851063829787234
0.3333333333333333
0.13559322033898305
1.148312893630994


##Pipeline

In [ ]:
tfidf_pipeline = pipeline.Pipeline([('tfidf vectorizer', tfidf_vectorizer), ('model', rf_model_tfidf)])

In [ ]:
tfidf_pipeline.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('tfidf vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=0.05, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 to...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=30, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,

In [ ]:
pipeline_train_predictions_tfidf = tfidf_pipeline.predict(train.text)
pipeline_test_predictions_tfidf = tfidf_pipeline.predict(test.text)

In [ ]:
print(accuracy_score(pipeline_train_predictions_tfidf, train.target))
print(accuracy_score(pipeline_test_predictions_tfidf, test.target))

0.986957939354418
0.9634941329856584


##Cross-validation (TF_IDF)

In [ ]:
cv_validation_tfidf = model_selection.cross_val_score(tfidf_pipeline,
                                train.text,
                                train.target,
                                scoring='accuracy',
                                cv=5)

In [ ]:
cv_validation_tfidf

array([0.96658517, 0.96495518, 0.96903015, 0.96740016, 0.96492659])

In [ ]:
print('Cross validation TFIDF accuracy score')
print(f'{cv_validation_tfidf.mean():.3f} +/- {cv_validation_tfidf.std():.3f}')

Cross validation TFIDF accuracy score
0.967 +/- 0.002


#Calculation for TEST

##Cleaning

In [ ]:
test = pd.read_csv("/content/drive/My Drive/Fraud_Finance_Data/News/test.csv")

In [ ]:
test['text_cleaning'] = test.text.progress_apply(preprocessing)


100%|██████████| 1599/1599 [00:34<00:00, 46.69it/s]


In [ ]:
test['text_union'] = test.text_cleaning.progress_apply(lambda x: " ".join(x))

100%|██████████| 1599/1599 [00:00<00:00, 87466.80it/s]


In [ ]:
test[['text', 'text_cleaning', 'text_union']].head(4)

,text,text_cleaning,text_union
0,Акции Match Group растут на 6.8% после включен...,"[акция, match, group, расти, на, 6.8, %, после...",акция match group расти на 6.8 % после включен...
1,8 сентября (ср) в 11:00 мск аналитик «АТОН» вс...,"[8, сентябрь, (, сравнить, ), в, 11, :, 00, мс...",8 сентябрь ( сравнить ) в 11 : 00 мск аналитик...
2,"​​ Как и обещали, мы добавили к видео про карт...","[​​, как, и, обещать, ,, мы, добавить, к, виде...","​​ как и обещать , мы добавить к видео про кар..."
3,None,[none],none


In [ ]:
test.head()

##TF-IDF Vectorizer

In [ ]:
# stopwords = set(nltk_stopwords.words('russian'))
# tfidf_vectorizer = TfidfVectorizer(min_df=0.05, stop_words=stopwords)
# processed_train_tfidf = tfidf_vectorizer.fit_transform(train.text_union)
processed_test_tfidf = tfidf_vectorizer.transform(test.text_union)
print(processed_test_tfidf[5])

  (0, 169)	0.08029028499473505
  (0, 168)	0.12906516490803066
  (0, 156)	0.1515484014347217
  (0, 154)	0.11664749764730831
  (0, 153)	0.10986611469893864
  (0, 130)	0.1830745413151203
  (0, 126)	0.09844138567922346
  (0, 125)	0.08597735966786565
  (0, 113)	0.25935794458834177
  (0, 112)	0.1199084684753696
  (0, 97)	0.12974778773125506
  (0, 94)	0.2722090109602271
  (0, 93)	0.15034674549774035
  (0, 82)	0.29304673835068495
  (0, 73)	0.08420742763954663
  (0, 67)	0.297224970742657
  (0, 66)	0.25867349552635416
  (0, 53)	0.12120709428090963
  (0, 44)	0.14895307781122222
  (0, 41)	0.2414298130367999
  (0, 27)	0.15034674549774035
  (0, 26)	0.5444247913669503
  (0, 21)	0.14170567224212205


##RandomForestClassifier

In [ ]:
# rf_model_tfidf = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=111, n_jobs=-1)
# rf_model_tfidf.fit(processed_test_tfidf, train.target)

train_predictions_tfidf = rf_model_tfidf.predict_proba(processed_train_tfidf)
test_predictions_tfidf = rf_model_tfidf.predict_proba(processed_test_tfidf)

###Export

In [ ]:
test_predictions_tfidf.shape

(1599, 2)

In [ ]:
test['proba'] = [i[1] for i in test_predictions_tfidf]
df_answer = test[['url','proba']]
df_answer.columns = ['url', 'Expected']

In [ ]:
df_answer.head(2)

,url,Expected
0,t.me/headlines_for_traders/9046,0.2
1,t.me/markettwits/155531,0.0


In [ ]:
df_answer.to_csv('/content/drive/MyDrive/Python ML CBR/df_answer_bezki_logreg.csv', index=False)

#Test1 - LSTM

In [ ]:
#Tokenizer
number_of_words = 10000

tokenizer = keras.preprocessing.text.Tokenizer(num_words = number_of_words)